In [ ]:
# Section 1: Imports and Configuration
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
import plotly.express as px
import configparser
import os

# Load configuration
config = configparser.ConfigParser()
config.read('config.ini')

# Google Sheets Configuration
SCOPES = config.get('Google', 'SCOPES')
GOOGLE_SHEET_ID = config.get('Google', 'GOOGLE_SHEET_ID')
GOOGLE_SHEET_RANGE = config.get('Google', 'GOOGLE_SHEET_RANGE')
GOOGLE_CREDENTIALS_PATH = config.get('Google', 'GOOGLE_CREDENTIALS_PATH')

# Shift Patterns Configuration
SHIFT_PATTERNS = {
    'standard': {'start': '08:00', 'end': '17:00'},
    'early': {'start': '06:00', 'end': '14:00'},
    'late': {'start': '14:00', 'end': '22:00'}
}

GROUP_SHIFTS = {
    'Dalmuir': 'standard',
    'Kilmalid': 'standard',
    'KB3': 'standard'  # Note: This is 'NOPS' in cask data
}

In [ ]:
# Section 2: Data Loading Functions

def load_json_data(file_path):
    """
    Load and parse the JSON biometric data
    Args:
        file_path: Path to JSON file
    Returns:
        DataFrame with parsed biometric data
    """
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
        
        # Create lists to store parsed data
        parsed_data = []
        
        # Iterate through each user's data
        for user_id, user_data in data.items():
            # Extract raw biometric data
            raw_data = user_data.get('historic_raw_data', [])
            
            # Convert to DataFrame format
            for record in raw_data:
                record['user_id'] = user_id
                parsed_data.append(record)
        
        # Create DataFrame
        df = pd.DataFrame(parsed_data)
        
        # Convert timestamp to datetime
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        
        return df
    
    except Exception as e:
        print(f"Error loading JSON data: {e}")
        return None

def load_google_sheet_data():
    """
    Load user mappings from Google Sheets
    Returns:
        DataFrame with user mappings
    """
    try:
        # Set up Google Sheets API
        creds = Credentials.from_authorized_user_file(GOOGLE_CREDENTIALS_PATH, SCOPES)
        service = build('sheets', 'v4', credentials=creds)
        
        # Call the Sheets API
        sheet = service.spreadsheets()
        result = sheet.values().get(
            spreadsheetId=GOOGLE_SHEET_ID,
            range=GOOGLE_SHEET_RANGE
        ).execute()
        
        # Convert to DataFrame
        values = result.get('values', [])
        if not values:
            print('No data found in Google Sheet')
            return None
            
        # Assuming first row contains headers
        headers = values[0]
        data = values[1:]
        df = pd.DataFrame(data, columns=headers)
        
        return df
    
    except Exception as e:
        print(f"Error loading Google Sheets data: {e}")
        return None

def load_cask_data(file_path):
    """
    Load cask movement data from CSV
    Args:
        file_path: Path to cask movements CSV
    Returns:
        DataFrame with cask movements
    """
    try:
        df = pd.read_csv(file_path)
        
        # Convert KB3 naming
        df['site'] = df['site'].replace('KB3', 'NOPS')
        
        # Create proper date from month and year
        df['date'] = pd.to_datetime(df[['year', 'month']].assign(day=1))
        
        # Convert percentage strings to floats
        for col in ['receipts_mom_var', 'dispatches_mom_var']:
            df[col] = df[col].str.rstrip('%').astype(float) / 100
            
        return df
    
    except Exception as e:
        print(f"Error loading cask data: {e}")
        return None

# Enhanced validation function with null handling
def validate_data_load():
    """
    Test data loading functions and print summaries, including null data analysis
    """
    # Load all data sources
    biometric_df = load_json_data('path_to_json')
    user_df = load_google_sheet_data()
    cask_df = load_cask_data('path_to_cask_csv')
    
    print("\nBiometric Data Summary:")
    if biometric_df is not None:
        print(f"Number of records: {len(biometric_df)}")
        print(f"Date range: {biometric_df['timestamp'].min()} to {biometric_df['timestamp'].max()}")
        print(f"Number of users: {biometric_df['user_id'].nunique()}")
        
        # Check for null values
        null_counts = biometric_df.isnull().sum()
        print("\nNull value counts:")
        print(null_counts[null_counts > 0])
        
        # Check for gaps in time series
        biometric_df['time_diff'] = biometric_df.groupby('user_id')['timestamp'].diff()
        large_gaps = biometric_df[biometric_df['time_diff'] > pd.Timedelta(hours=1)]
        if not large_gaps.empty:
            print("\nFound gaps in data collection (>1 hour):")
            print(f"Number of gaps: {len(large_gaps)}")
            print("Sample gaps:")
            print(large_gaps.head())
        
    print("\nUser Mapping Summary:")
    if user_df is not None:
        print(f"Number of users: {len(user_df)}")
        print("Groups represented:", user_df['group'].unique())
        # Check for missing values in crucial columns
        missing_values = user_df[user_df.isnull().any(axis=1)]
        if not missing_values.empty:
            print("\nWarning: Found incomplete user records:")
            print(missing_values)
        
    print("\nCask Movement Summary:")
    if cask_df is not None:
        print(f"Date range: {cask_df['date'].min()} to {cask_df['date'].max()}")
        print("Sites represented:", cask_df['site'].unique())
        # Check for missing monthly data
        expected_months = pd.date_range(cask_df['date'].min(), cask_df['date'].max(), freq='M')
        for site in cask_df['site'].unique():
            site_data = cask_df[cask_df['site'] == site]
            missing_months = [month for month in expected_months 
                            if month.strftime("%Y-%m") not in site_data['date'].dt.strftime("%Y-%m").values]
            if missing_months:
                print(f"\nMissing months for {site}:")
                print(missing_months)
                

# Section 3: Data Cleaning and Processing Functions

In [ ]:
# Section 3: Data Cleaning and Processing Functions

def clean_biometric_data(df, user_mapping_df):
    """
    Clean and structure biometric data
    Args:
        df: Raw biometric DataFrame
        user_mapping_df: User mapping DataFrame with group information
    Returns:
        Cleaned DataFrame with working hours filter applied
    """
    # Create copy to avoid modifying original
    cleaned_df = df.copy()
    
    # Merge with user mapping to get group information
    cleaned_df = cleaned_df.merge(user_mapping_df[['user_id', 'group', 'org']], 
                                on='user_id', 
                                how='left')
    
    # Filter for PR organization
    cleaned_df = cleaned_df[cleaned_df['org'] == 'PR']
    
    # Apply shift patterns
    def is_working_hours(row):
        shift = SHIFT_PATTERNS[GROUP_SHIFTS[row['group']]]
        time = row['timestamp'].time()
        start = datetime.strptime(shift['start'], '%H:%M').time()
        end = datetime.strptime(shift['end'], '%H:%M').time()
        return start <= time <= end
    
    cleaned_df['is_working_hours'] = cleaned_df.apply(is_working_hours, axis=1)
    
    # Handle missing values
    cleaned_df['heart_rate'] = cleaned_df['heart_rate'].fillna(method='ffill', limit=3)  # Forward fill up to 3 periods
    
    # Create time-based features
    cleaned_df['hour'] = cleaned_df['timestamp'].dt.hour
    cleaned_df['day_of_week'] = cleaned_df['timestamp'].dt.day_name()
    cleaned_df['week'] = cleaned_df['timestamp'].dt.isocalendar().week
    cleaned_df['month'] = cleaned_df['timestamp'].dt.month
    
    # Flag sparse data periods
    def flag_sparse_data(group_df):
        expected_records = 12  # Assuming 5-minute intervals
        actual_records = len(group_df)
        group_df['data_sparse'] = actual_records < expected_records
        return group_df
    
    cleaned_df = cleaned_df.groupby(['user_id', 'timestamp'].dt.floor('H')).apply(flag_sparse_data)
    
    return cleaned_df

def process_cask_data(df):
    """
    Process cask movement data for analysis
    Args:
        df: Raw cask movement DataFrame
    Returns:
        Processed DataFrame with additional metrics
    """
    processed_df = df.copy()
    
    # Calculate efficiency metrics
    processed_df['dispatch_receipt_ratio'] = processed_df['dispatches'] / processed_df['receipts']
    
    # Calculate rolling averages
    processed_df['rolling_receipts'] = processed_df.groupby('site')['receipts'].rolling(window=3, min_periods=1).mean().reset_index(0, drop=True)
    processed_df['rolling_dispatches'] = processed_df.groupby('site')['dispatches'].rolling(window=3, min_periods=1).mean().reset_index(0, drop=True)
    
    return processed_df

def create_analysis_dataset(biometric_df, cask_df):
    """
    Combine biometric and cask data for analysis
    Args:
        biometric_df: Cleaned biometric DataFrame
        cask_df: Processed cask DataFrame
    Returns:
        Combined DataFrame ready for analysis
    """
    # Aggregate biometric data to daily level
    daily_biometric = biometric_df[biometric_df['is_working_hours']].groupby(
        ['group', biometric_df['timestamp'].dt.date]
    ).agg({
        'heart_rate': ['mean', 'std', 'min', 'max'],
        'data_sparse': 'sum',
        'user_id': 'nunique'  # Number of users contributing data
    }).reset_index()
    
    # Rename columns for clarity
    daily_biometric.columns = ['group', 'date', 'heart_rate_mean', 'heart_rate_std', 
                             'heart_rate_min', 'heart_rate_max', 'sparse_hours', 'active_users']
    
    # Convert date to datetime for merging
    daily_biometric['date'] = pd.to_datetime(daily_biometric['date'])
    
    # Merge with cask data
    analysis_df = daily_biometric.merge(
        cask_df,
        left_on=['group', daily_biometric['date'].dt.to_period('M')],
        right_on=['site', cask_df['date'].dt.to_period('M')],
        how='outer'
    )
    
    return analysis_df

# Section 4: Analysis Functions

In [ ]:
# Section 4: Analysis Functions

def analyze_group_patterns(analysis_df):
    """
    Analyze patterns within and between groups
    Args:
        analysis_df: Combined analysis DataFrame
    Returns:
        Dictionary containing analysis results
    """
    results = {}
    
    # Daily patterns
    daily_patterns = analysis_df.groupby(['group', 'hour'])[['heart_rate_mean']].mean()
    daily_variation = analysis_df.groupby(['group', 'hour'])[['heart_rate_mean']].std()
    
    results['daily_patterns'] = {
        'mean_patterns': daily_patterns,
        'variations': daily_variation
    }
    
    # Weekly patterns
    weekly_patterns = analysis_df.groupby(['group', 'day_of_week'])[
        ['receipts', 'dispatches', 'heart_rate_mean']
    ].mean()
    
    results['weekly_patterns'] = weekly_patterns
    
    # Monthly trends
    monthly_trends = analysis_df.groupby(['group', 'month'])[
        ['receipts', 'dispatches', 'heart_rate_mean']
    ].agg(['mean', 'std'])
    
    results['monthly_trends'] = monthly_trends
    
    return results

def analyze_group_differences(analysis_df):
    """
    Statistical analysis of differences between groups
    Args:
        analysis_df: Combined analysis DataFrame
    Returns:
        Dictionary containing statistical test results
    """
    from scipy import stats
    
    results = {}
    
    # Function for pairwise group comparisons
    def pairwise_comparison(data, metric):
        groups = data['group'].unique()
        comparisons = []
        
        for i in range(len(groups)):
            for j in range(i+1, len(groups)):
                group1, group2 = groups[i], groups[j]
                stat, pval = stats.ttest_ind(
                    data[data['group'] == group1][metric].dropna(),
                    data[data['group'] == group2][metric].dropna()
                )
                comparisons.append({
                    'group1': group1,
                    'group2': group2,
                    'statistic': stat,
                    'p_value': pval
                })
        
        return comparisons
    
    # Compare key metrics
    metrics = ['heart_rate_mean', 'receipts', 'dispatches', 'dispatch_receipt_ratio']
    
    for metric in metrics:
        results[f'{metric}_comparisons'] = pairwise_comparison(analysis_df, metric)
    
    # Identify outlier groups
    def identify_outliers(data, metric):
        z_scores = stats.zscore(data.groupby('group')[metric].mean())
        return pd.Series(z_scores, name='z_score')
    
    for metric in metrics:
        results[f'{metric}_outliers'] = identify_outliers(analysis_df, metric)
    
    return results

def analyze_productivity_correlations(analysis_df):
    """
    Analyze correlations between biometric data and productivity metrics
    Args:
        analysis_df: Combined analysis DataFrame
    Returns:
        Dictionary containing correlation analyses
    """
    results = {}
    
    # Overall correlations
    biometric_cols = ['heart_rate_mean', 'heart_rate_std']
    productivity_cols = ['receipts', 'dispatches', 'dispatch_receipt_ratio']
    
    correlation_matrix = analysis_df[biometric_cols + productivity_cols].corr()
    results['overall_correlations'] = correlation_matrix
    
    # Group-specific correlations
    group_correlations = {}
    for group in analysis_df['group'].unique():
        group_data = analysis_df[analysis_df['group'] == group]
        group_correlations[group] = group_data[biometric_cols + productivity_cols].corr()
    
    results['group_correlations'] = group_correlations
    
    # Time-lagged correlations
    def calculate_lagged_correlation(data, col1, col2, max_lag=3):
        lagged_corrs = []
        for lag in range(max_lag + 1):
            corr = data[col1].corr(data[col2].shift(lag))
            lagged_corrs.append({'lag': lag, 'correlation': corr})
        return lagged_corrs
    
    lagged_correlations = {}
    for bio_col in biometric_cols:
        for prod_col in productivity_cols:
            key = f'{bio_col}_vs_{prod_col}'
            lagged_correlations[key] = calculate_lagged_correlation(
                analysis_df, bio_col, prod_col
            )
    
    results['lagged_correlations'] = lagged_correlations
    
    return results

def analyze_data_quality(analysis_df):
    """
    Analyze data quality and coverage
    Args:
        analysis_df: Combined analysis DataFrame
    Returns:
        Dictionary containing data quality metrics
    """
    results = {}
    
    # Data coverage by group
    coverage = analysis_df.groupby('group').agg({
        'sparse_hours': 'sum',
        'active_users': ['mean', 'min', 'max']
    })
    
    results['group_coverage'] = coverage
    
    # Identify periods of low data quality
    low_quality_periods = analysis_df[
        (analysis_df['sparse_hours'] > 4) |  # More than 4 hours of sparse data
        (analysis_df['active_users'] < 3)     # Less than 3 active users
    ]
    
    results['low_quality_periods'] = low_quality_periods
    
    return results

# Section 5: Visualization Functions

In [ ]:


def create_daily_pattern_plots(analysis_results):
    """
    Create visualizations of daily patterns
    Args:
        analysis_results: Results from analyze_group_patterns
    Returns:
        Dictionary of plotly figures
    """
    figures = {}
    
    # Heart rate patterns throughout the day
    daily_patterns = analysis_results['daily_patterns']['mean_patterns'].reset_index()
    
    fig_heart_rate = px.line(
        daily_patterns,
        x='hour',
        y='heart_rate_mean',
        color='group',
        title='Average Heart Rate Throughout Working Hours',
        labels={'heart_rate_mean': 'Average Heart Rate', 'hour': 'Hour of Day'}
    )
    fig_heart_rate.update_layout(
        hovermode='x unified',
        showlegend=True
    )
    
    figures['daily_heart_rate'] = fig_heart_rate
    
    return figures

def create_productivity_plots(analysis_df):
    """
    Create visualizations of productivity metrics
    Args:
        analysis_df: Combined analysis DataFrame
    Returns:
        Dictionary of plotly figures
    """
    figures = {}
    
    # Monthly productivity trends
    fig_productivity = px.line(
        analysis_df,
        x='date',
        y=['receipts', 'dispatches'],
        color='group',
        title='Monthly Productivity Metrics',
        facet_col='variable',
        labels={'value': 'Count', 'date': 'Month'}
    )
    fig_productivity.update_layout(
        hovermode='x unified',
        showlegend=True
    )
    
    figures['monthly_productivity'] = fig_productivity
    
    # Efficiency ratio plot
    fig_efficiency = px.line(
        analysis_df,
        x='date',
        y='dispatch_receipt_ratio',
        color='group',
        title='Dispatch/Receipt Efficiency Ratio',
        labels={'dispatch_receipt_ratio': 'Efficiency Ratio', 'date': 'Month'}
    )
    
    figures['efficiency_ratio'] = fig_efficiency
    
    return figures

def create_correlation_plots(analysis_results):
    """
    Create correlation visualizations
    Args:
        analysis_results: Results from analyze_productivity_correlations
    Returns:
        Dictionary of plotly figures
    """
    figures = {}
    
    # Overall correlation heatmap
    corr_matrix = analysis_results['overall_correlations']
    fig_corr = px.imshow(
        corr_matrix,
        title='Correlation Matrix: Biometrics vs Productivity',
        labels=dict(color="Correlation"),
        color_continuous_scale='RdBu'
    )
    
    figures['correlation_matrix'] = fig_corr
    
    # Group-specific correlation comparison
    group_corrs = analysis_results['group_correlations']
    # Create subplot for each group
    from plotly.subplots import make_subplots
    fig_group_corr = make_subplots(
        rows=1, 
        cols=len(group_corrs),
        subplot_titles=list(group_corrs.keys())
    )
    
    for i, (group, corr) in enumerate(group_corrs.items(), 1):
        fig_group_corr.add_trace(
            px.imshow(corr).data[0],
            row=1, col=i
        )
    
    figures['group_correlations'] = fig_group_corr
    
    return figures

def create_data_quality_plots(analysis_results):
    """
    Create visualizations of data quality metrics
    Args:
        analysis_results: Results from analyze_data_quality
    Returns:
        Dictionary of plotly figures
    """
    figures = {}
    
    # Data coverage by group
    coverage = analysis_results['group_coverage'].reset_index()
    fig_coverage = px.bar(
        coverage,
        x='group',
        y=['sparse_hours', 'active_users'],
        title='Data Quality Metrics by Group',
        barmode='group'
    )
    
    figures['data_quality'] = fig_coverage
    
    # Timeline of low quality periods
    low_quality = analysis_results['low_quality_periods']
    fig_quality_timeline = px.scatter(
        low_quality,
        x='date',
        y='group',
        size='sparse_hours',
        color='active_users',
        title='Periods of Low Data Quality',
        labels={'sparse_hours': 'Hours of Sparse Data', 'active_users': 'Active Users'}
    )
    
    figures['quality_timeline'] = fig_quality_timeline
    
    return figures

def create_interactive_dashboard(analysis_df, analysis_results):
    """
    Create an interactive dashboard combining key visualizations
    Args:
        analysis_df: Combined analysis DataFrame
        analysis_results: Dictionary of analysis results
    Returns:
        Plotly figure containing dashboard
    """
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
    
    # Create main dashboard figure
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=(
            'Daily Heart Rate Patterns',
            'Monthly Productivity',
            'Efficiency Ratio',
            'Data Quality',
            'Correlation Matrix',
            'Group Comparison'
        )
    )
    
    # Add traces from individual plots
    daily_patterns = create_daily_pattern_plots(analysis_results)['daily_heart_rate']
    productivity_plots = create_productivity_plots(analysis_df)
    correlation_plots = create_correlation_plots(analysis_results)
    quality_plots = create_data_quality_plots(analysis_results)
    
    # Combine all plots into dashboard
    # (Add specific trace copying code here)
    
    fig.update_layout(height=1200, width=1600, title_text="Group Analysis Dashboard")
    
    return fig

# Section 6: Main Execution


In [ ]:

def main():
    """
    Main execution flow for data processing and analysis
    """
    print("Starting analysis pipeline...")
    
    # 1. Load all data sources
    print("\nLoading data...")
    biometric_df = load_json_data('path_to_json')
    user_df = load_google_sheet_data()
    cask_df = load_cask_data('path_to_cask_csv')
    
    # Validate data loading
    validate_data_load()
    
    # 2. Clean and process data
    print("\nProcessing data...")
    cleaned_biometric = clean_biometric_data(biometric_df, user_df)
    processed_cask = process_cask_data(cask_df)
    
    # 3. Create combined analysis dataset
    print("\nCreating analysis dataset...")
    analysis_df = create_analysis_dataset(cleaned_biometric, processed_cask)
    
    # 4. Run analyses
    print("\nRunning analyses...")
    pattern_results = analyze_group_patterns(analysis_df)
    difference_results = analyze_group_differences(analysis_df)
    correlation_results = analyze_productivity_correlations(analysis_df)
    quality_results = analyze_data_quality(analysis_df)
    
    # Combine all results
    analysis_results = {
        'patterns': pattern_results,
        'differences': difference_results,
        'correlations': correlation_results,
        'quality': quality_results
    }
    
    # 5. Create visualizations
    print("\nGenerating visualizations...")
    daily_plots = create_daily_pattern_plots(pattern_results)
    productivity_plots = create_productivity_plots(analysis_df)
    correlation_plots = create_correlation_plots(correlation_results)
    quality_plots = create_data_quality_plots(quality_results)
    
    # 6. Create dashboard
    print("\nCreating dashboard...")
    dashboard = create_interactive_dashboard(analysis_df, analysis_results)
    
    # 7. Save results
    print("\nSaving results...")
    
    # Save processed data
    analysis_df.to_csv('outputs/processed_analysis_data.csv', index=False)
    
    # Save plots as HTML files
    for name, fig in daily_plots.items():
        fig.write_html(f'outputs/plots/{name}.html')
    for name, fig in productivity_plots.items():
        fig.write_html(f'outputs/plots/{name}.html')
    for name, fig in correlation_plots.items():
        fig.write_html(f'outputs/plots/{name}.html')
    for name, fig in quality_plots.items():
        fig.write_html(f'outputs/plots/{name}.html')
    
    # Save dashboard
    dashboard.write_html('outputs/dashboard.html')
    
    # 8. Generate summary report
    print("\nGenerating summary report...")
    generate_summary_report(analysis_results, analysis_df)
    
    print("\nAnalysis complete! Results saved in 'outputs' directory.")
    
    return analysis_df, analysis_results, dashboard

def generate_summary_report(analysis_results, analysis_df):
    """
    Generate a summary report of key findings
    """
    with open('outputs/summary_report.txt', 'w') as f:
        f.write("Analysis Summary Report\n")
        f.write("=====================\n\n")
        
        # Data coverage
        f.write("Data Coverage:\n")
        f.write("--------------\n")
        coverage = analysis_results['quality']['group_coverage']
        f.write(f"{coverage.to_string()}\n\n")
        
        # Key findings
        f.write("Key Findings:\n")
        f.write("-------------\n")
        
        # Group differences
        differences = analysis_results['differences']
        for metric, comparisons in differences.items():
            if metric.endswith('_comparisons'):
                f.write(f"\n{metric.replace('_comparisons', '').title()} Comparisons:\n")
                for comp in comparisons:
                    if comp['p_value'] < 0.05:
                        f.write(f"- Significant difference between {comp['group1']} and {comp['group2']}\n")
        
        # Correlations
        f.write("\nStrong Correlations:\n")
        corr_matrix = analysis_results['correlations']['overall_correlations']
        strong_corrs = [(i, j, corr_matrix.loc[i,j]) 
                       for i in corr_matrix.index 
                       for j in corr_matrix.columns 
                       if abs(corr_matrix.loc[i,j]) > 0.7 and i != j]
        for i, j, corr in strong_corrs:
            f.write(f"- {i} vs {j}: {corr:.2f}\n")

if __name__ == "__main__":
    try:
        # Create outputs directory if it doesn't exist
        os.makedirs('outputs/plots', exist_ok=True)
        
        # Run main analysis
        analysis_df, analysis_results, dashboard = main()
        
    except Exception as e:
        print(f"Error in main execution: {e}")
        raise